In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import requests
from io import StringIO 
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
df1 = pd.read_csv("/content/drive/My Drive/PDDA/Data3_1_2020/train.csv")
df2 = pd.read_csv("/content/drive/My Drive/PDDA/Data3_1_2020/test.csv")
valid = pd.read_csv("/content/drive/My Drive/PDDA/Part_Of_Expected_Results/real_result_20perc.csv")

In [0]:
# remove all rows that contains missing value
df1.replace(['-999', -999], np.nan, inplace=True)
df1.dropna(axis=0, inplace=True)

df2.replace(['-999', -999], np.nan, inplace=True)
df2.dropna(axis=0, inplace=True)

In [0]:
# seperate the features and targets
X=df1.drop(["DTC","DTS",],axis =1)
y = df1.DTC

In [0]:
test = valid.drop(['DTC','DTS'],axis=1)

In [0]:
print(y[:5][:])

[[125.9611 306.8676]
 [125.9726 305.8347]
 [125.8528 305.9089]
 [125.7362 305.1737]
 [125.9774 304.4476]]


In [0]:
y.shape

(20525, 2)

In [0]:
!pip install bayesian-optimization

In [0]:
import lightgbm as lgb
import catboost as cgb
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

In [0]:

# Define R-Sqaured
def xgb_r2(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(preds, labels)
  
dtrain = xgb.DMatrix(X, y, feature_names=X.columns.values)

# Define Objective Function
def hyp_xgb(max_depth, subsample, colsample_bytree,min_child_weight, gamma ):
    params = {
    'n_estimators': 500,
    'eta': 0.05,
    'objective': 'reg:linear',
    'eval_metric':'rmse', # Optional --> Use eval_metric if you want to stop evaluation based on eval_metric 
    'silent': 1
     }
    params['max_depth'] = int(round(max_depth))
    params['subsample'] = max(min(subsample, 1), 0)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['min_child_weight'] = int(min_child_weight)
    params['gamma'] = max(gamma, 0)
    scores = xgb.cv(params, dtrain, num_boost_round=1000,verbose_eval=False, early_stopping_rounds=10, feval=xgb_r2, maximize=True, nfold=5)
    return  scores['test-r2-mean'].iloc[-1]

In [0]:

pds ={
  'min_child_weight':(14, 20),
  'gamma':(0, 5),
  'subsample':(0.5, 1),
  'colsample_bytree':(0.1, 1),
  'max_depth': (5, 10)
}

In [0]:
# Surrogate model

optimizer = BayesianOptimization(hyp_xgb, pds, random_state=1234)
                                  
# Optimize

optimizer.maximize(init_points=5, n_iter=20)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------
|  1        | -1.024e+0 |  0.2724   |  3.111    |  7.189    |  18.71    |  0.89     |
|  2        | -5.733e+0 |  0.3453   |  1.382    |  9.009    |  19.75    |  0.938    |
|  3        | -5.747e+0 |  0.422    |  2.505    |  8.417    |  18.28    |  0.6851   |
|  4        | -5.67e+03 |  0.6051   |  2.515    |  5.069    |  18.64    |  0.9413   |
|  5        | -5.737e+0 |  0.4284   |  3.077    |  5.377    |  16.21    |  0.9666   |
|  6        | -5.737e+0 |  0.4284   |  3.077    |  5.377    |  16.21    |  0.9666   |
|  7        | -5.714e+0 |  0.4442   |  4.107    |  7.025    |  18.49    |  0.8878   |
|  8        | -1.024e+0 |  0.1999   |  1.093    |  8.358    |  19.16    |  0.8987   |
|  9        | -5.666e+0 |  0.8486   |  4.032    |  5.49     |  16.28    |  0.9397   |
|  10       | -5.644e+0 |  0.8625   |  1.452    |  8.4

# Implementing Models

In [0]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [0]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [0]:
best_xgb_model = xgb.XGBRegressor(colsample_bytree=0.8606,
                 gamma=4.794,                 
                 learning_rate=0.07,
                 max_depth=7,
                 min_child_weight=14.03,
                 n_estimators=1000,                                                                    
                 subsample=0.9863,
                 seed=1234)

In [0]:
best_xgb_model.fit(X,y)

[19:07:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8606, gamma=4.794,
             importance_type='gain', learning_rate=0.07, max_delta_step=0,
             max_depth=7, min_child_weight=14.03, missing=None,
             n_estimators=1000, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=1234, silent=None, subsample=0.9863, verbosity=1)

In [0]:
cat_DTC_preds = best_xgb_model.predict(test)

# Checking model performance with Validation Data

In [0]:
real_DTC_values = valid['DTC']

In [0]:
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTC_values, cat_DTC_preds))))

Root Mean Square Error is: 4.61911


# Making Predictions for DTS

In [0]:
X=df1.drop(["DTC","DTS",],axis =1)
y = df1.DTS

In [0]:
best_xgb_model.fit(X,y)

[19:11:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8606, gamma=4.794,
             importance_type='gain', learning_rate=0.07, max_delta_step=0,
             max_depth=7, min_child_weight=14.03, missing=None,
             n_estimators=1000, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=1234, silent=None, subsample=0.9863, verbosity=1)

In [0]:
cat_DTS_preds = best_xgb_model.predict(test)

# Checking Model performance on DTS

In [0]:
real_DTS_values = valid['DTS']

In [0]:
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTS_values, cat_DTS_preds))))

Root Mean Square Error is: 24.32433


# Checking Overall Performance 

In [0]:
results = pd.DataFrame(columns=['DTC','DTS'])

In [0]:
results['DTC'] = cat_DTC_preds
results['DTS'] = cat_DTS_preds
results.head()

,DTC,DTS
0,68.681435,126.749466
1,93.896721,175.521484
2,87.239418,182.808304
3,77.822433,155.265884
4,77.800446,151.914185


In [0]:
results_data = np.array(results)

In [0]:
valid_data = np.array(valid)
y_real = valid_data[:,-2:]

In [0]:
 print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(y_real, results))))

Root Mean Square Error is: 17.50727


# Trying out gradient boosting

#Predictiing DTC

In [0]:
# seperate the features and targets
X=df1.drop(["DTC","DTS",],axis =1)
y = df1.DTC

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
# train test split
# please remember to use random_state for all randomization steps, this will ensure we get the same results 
# as yours during the validation phase. 
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1 ,random_state=1234)


In [0]:
from sklearn.ensemble import GradientBoostingRegressor

In [0]:
regressor = GradientBoostingRegressor(
    max_depth=2,
    n_estimators=3,
    learning_rate=0.05
)
regressor.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
errors = [mean_squared_error(y_test, y_pred) for y_pred in regressor.staged_predict(X_test)]
best_n_estimators = np.argmin(errors)

In [0]:
best_regressor = GradientBoostingRegressor(
    max_depth=2,
    n_estimators=best_n_estimators,
    learning_rate=1.0
)
best_regressor.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=2,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
reg_pred = best_regressor.predict(X_test)

In [0]:
from sklearn import metrics

In [0]:
print('MAE:', metrics.mean_absolute_error(y_test,reg_pred))
print('MSE:',metrics.mean_squared_error(y_test,reg_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test,reg_pred)))

MAE: 4.655082243647228
MSE: 42.067948526974995
RMSE: 6.485980922495455


In [0]:
cat_DTC_preds = best_regressor.predict(test)

In [0]:
real_DTC_values = valid['DTC']

In [0]:
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTC_values, cat_DTC_preds))))

Root Mean Square Error is: 6.16864


# Making DTS Prediction

In [0]:
X=df1.drop(["DTC","DTS",],axis =1)
y = df1.DTS

In [0]:
# train test split
# please remember to use random_state for all randomization steps, this will ensure we get the same results 
# as yours during the validation phase. 
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1 ,random_state=1234)


In [0]:
regressor = GradientBoostingRegressor(
    max_depth=2,
    n_estimators=3,
    learning_rate=0.05
)
regressor.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
errors = [mean_squared_error(y_test, y_pred) for y_pred in regressor.staged_predict(X_test)]
best_n_estimators = np.argmin(errors)

In [0]:
best_regressor = GradientBoostingRegressor(
    max_depth=2,
    n_estimators=best_n_estimators,
    learning_rate=1.0
)
best_regressor.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=2,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
reg_pred = best_regressor.predict(X_test)

In [0]:
print('MAE:', metrics.mean_absolute_error(y_test,reg_pred))
print('MSE:',metrics.mean_squared_error(y_test,reg_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test,reg_pred)))

MAE: 12.54505603451885
MSE: 356.0922587536634
RMSE: 18.87040695781793


# Checking model performance on DTS

In [0]:
cat_DTS_preds = best_regressor.predict(test)

In [0]:
real_DTS_values = valid['DTS']

In [0]:
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTS_values, cat_DTS_preds))))

Root Mean Square Error is: 28.87236


# Checking Overall Gradient Boosting Model Performance

In [0]:
results = pd.DataFrame(columns=['DTC','DTS'])

In [0]:
results['DTC'] = cat_DTC_preds
results['DTS'] = cat_DTS_preds
results.head()

,DTC,DTS
0,73.703677,133.283763
1,90.322720,202.983293
2,90.322720,188.590044
3,73.703677,133.283763
4,73.703677,133.283763


In [0]:
results_data = np.array(results)

In [0]:
valid_data = np.array(valid)
y_real = valid_data[:,-2:]

In [0]:
 print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(y_real, results))))

Root Mean Square Error is: 20.87661
